In [2]:
import configparser

# Read Config
config = configparser.ConfigParser()
config.read('loadpg_config.properties')
SPARK_MASTER = config.get('loadpg', 'spark_master')
SPARK_DRIVER_HOST = config.get('loadpg', 'spark_driver_host')
SPARK_DRIVER_BINDADDRES = config.get('loadpg', 'spark_driver_bindaddress')
HDFS_URL = config.get('loadpg', 'hdfs_url')
HADOOP_USER_NAME = config.get('loadpg', 'hadoop_user_name')
PG_DRIVER_JAR_PATH = config.get('loadpg', 'pg_driver_jar_path')
PG_IP = config.get('loadpg', 'pg_ip')
PG_USER = config.get('loadpg', 'pg_user')
PG_PASS = config.get('loadpg', 'pg_pass')

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, ArrayType, IntegerType, DoubleType
from pyspark.sql.functions import udf, col
import os
os.environ["HADOOP_USER_NAME"] = HADOOP_USER_NAME

In [3]:
spark = SparkSession.builder \
    .appName("Load_to_Postgres") \
    .config("spark.master", SPARK_MASTER) \
    .config("spark.driver.host", SPARK_DRIVER_HOST) \
    .config("spark.driver.bindAddress", SPARK_DRIVER_BINDADDRES) \
    .config("spark.hadoop.fs.defaultFS", f'hdfs://{HDFS_URL}/') \
    .config("spark.executor.memory", "2G") \
    .config("spark.hadoop.parquet.block.size", "16777216") \
    .config("spark.hadoop.dfs.blocksize", "16777216") \
    .config("spark.hadoop.dfs.replication", "1") \
    .config("spark.sql.files.maxPartitionBytes", "134217728") \
    .config("spark.memory.fraction", "0.1") \
    .config("spark.jars", PG_DRIVER_JAR_PATH) \
    .getOrCreate()

24/05/20 21:38:36 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [ ]:
sc = spark.sparkContext
hadoop = sc._jvm.org.apache.hadoop
fs = hadoop.fs.FileSystem
conf = hadoop.conf.Configuration()
conf.set("fs.defaultFS", f'hdfs://{HDFS_URL}/')
path = hadoop.fs.Path('/wiki_out5')

for f in fs.get(conf).listStatus(path):
    print(str(f.getPath()))

### Load Doc Details to Postgres

In [4]:
hdfs_input = f'hdfs://{HDFS_URL}/wiki2'
df = spark.read.parquet(hdfs_input).select(col('id').alias('doc_id'), 'title', 'url')

In [5]:
df.printSchema()

root
 |-- doc_id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- url: string (nullable = true)



In [7]:
df.write.format("jdbc") \
    .option("url", f"jdbc:postgresql://{PG_IP}:5432/db1") \
    .option("driver", "org.postgresql.Driver").option("dbtable", "doc_det2") \
    .option("user", PG_USER) \
    .option("password", PG_PASS) \
    .save()

### Load TF-IDF to Postgres
Note: Ingesting parallelly like the one done in this section will result in the data not being sorted physically in the destination database despite the source being sorted already

In [ ]:
hdfs_input = f'hdfs://{HDFS_URL}/wiki_out5'
df = spark.read.parquet(hdfs_input)

In [5]:
df.printSchema()

root
 |-- term: string (nullable = true)
 |-- term_tfidf_array: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- doc_id: string (nullable = true)
 |    |    |-- tfidf: double (nullable = true)



In [ ]:
def f_parse_structarr(structarr):
    l1 = []
    for x in structarr:
        l1.append(f'"({x.doc_id},{x.tfidf})"')
    return '{' + ','.join(l1) + '}'

udf_parse_structarr = udf(f_parse_structarr, StringType())

df_string = df.select(col("term"), udf_parse_structarr("term_tfidf_array").alias('docs'))

In [ ]:
df_string.write.format("jdbc") \
    .option("url", f"jdbc:postgresql://{PG_IP}:5432/db1") \
    .option("driver", "org.postgresql.Driver").option("dbtable", "term_docs2") \
    .option("user", PG_USER) \
    .option("password", PG_PASS) \
    .save()